In [1]:
import os
from dotenv import load_dotenv

# Load variables from the .env file into the environment
load_dotenv()

# Now you can access them using os.getenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time # Good practice for potential rate limits

# IF using environment variables (recommended):
CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')

# Rest of your code...
if not CLIENT_ID or not CLIENT_SECRET:
    print("Error: Spotify Client ID or Secret not found.")
    print("Please check your .env file and ensure it's loaded correctly.")
    # It's better to exit or raise an error here if critical
    # exit()
else:
     print("Successfully loaded environment variables.") # Add a confirmation

# Authenticate using Client Credentials Flow
try:
    auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    print("Successfully authenticated with Spotify.")
except Exception as e:
    print(f"Error authenticating with Spotify: {e}")
    # exit()

Successfully loaded environment variables.
Successfully authenticated with Spotify.


In [2]:

def get_playlist_tracks(playlist_id):
    """Fetches all tracks from a Spotify playlist, handling pagination."""
    all_tracks_data = []
    offset = 0
    limit = 100 # Max items per request

    print(f"Fetching tracks for playlist ID: {playlist_id}")

    while True:
        try:
            # Make the API call
            response = sp.playlist_items(
                playlist_id,
                fields='items(track(name, artists(name), album(name))), next', # Specify only needed fields
                additional_types=['track'], # Ensure we only get tracks
                offset=offset,
                limit=limit
            )

            if not response or 'items' not in response:
                print("Warning: Received empty or invalid response.")
                break

            items = response.get('items', [])
            if not items:
                print("No more items found.")
                break # Exit loop if no items are returned

            for item in items:
                track_info = item.get('track')
                if track_info: # Ensure track data exists
                    track_name = track_info.get('name', 'N/A')
                    # Handle multiple artists
                    artist_names = ', '.join([artist.get('name', 'N/A') for artist in track_info.get('artists', [])])
                    album_name = track_info.get('album', {}).get('name', 'N/A')

                    all_tracks_data.append({
                        'name': track_name,
                        'artist': artist_names,
                        'album': album_name
                    })
                else:
                     print(f"Skipping item without track data: {item}")


            # Check if there are more pages
            if response.get('next'):
                offset += limit # Move to the next page
                print(f"Fetching next batch (offset: {offset})...")
                time.sleep(0.5) # Add a small delay to be kind to the API
            else:
                print("Reached the end of the playlist.")
                break # No more pages

        except spotipy.exceptions.SpotifyException as e:
             print(f"Spotify API Error: {e.http_status} - {e.msg}")
             # Handle specific errors like rate limiting (429) if needed
             if e.http_status == 429:
                 retry_after = int(e.headers.get('Retry-After', 5)) # Get retry time from header
                 print(f"Rate limited. Retrying after {retry_after} seconds...")
                 time.sleep(retry_after)
             else:
                 break # Break on other errors
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            break

    print(f"Fetched {len(all_tracks_data)} tracks total.")
    return all_tracks_data

PLAYLIST_ID = '1tiiLc9YZLYmjMQ7ADlPXZ'
# --- Fetch the data ---
playlist_songs = get_playlist_tracks(PLAYLIST_ID)
# --- End Fetching Data ---

# --- Display Results (Example) ---
if playlist_songs:
    print("\n--- Playlist Songs ---")
    for i, song in enumerate(playlist_songs):
        print(f"{i+1}. Name: {song['name']}, Artist: {song['artist']}, Album: {song['album']}")
    print("----------------------")
else:
    print("Could not retrieve any songs from the playlist.")

Fetching tracks for playlist ID: 1tiiLc9YZLYmjMQ7ADlPXZ
Reached the end of the playlist.
Fetched 37 tracks total.

--- Playlist Songs ---
1. Name: We Will Rock You - Remastered 2011, Artist: Queen, Album: News Of The World (2011 Remaster)
2. Name: Cane Shuga, Artist: Glass Animals, Album: How To Be A Human Being
3. Name: Young Folks, Artist: Peter Bjorn and John, Album: Writer's Block
4. Name: Feel It Still, Artist: Portugal. The Man, Album: Woodstock
5. Name: Lovers Rock, Artist: TV Girl, Album: French Exit
6. Name: Cigarettes out the Window, Artist: TV Girl, Album: Who Really Cares
7. Name: Kitana, Artist: Princess Nokia, Album: 1992 Deluxe
8. Name: Ribs, Artist: Lorde, Album: Pure Heroine
9. Name: Cosmic Love, Artist: Florence + The Machine, Album: Lungs (Deluxe Version)
10. Name: Florida!!! (feat. Florence + The Machine), Artist: Taylor Swift, Florence + The Machine, Album: THE TORTURED POETS DEPARTMENT
11. Name: Kiss With A Fist, Artist: Florence + The Machine, Album: Lungs (Delux

In [ ]:
pip install --upgrade google-generativeai

In [ ]:
import google.generativeai as genai
import os
import time
import re # Import regular expressions for parsing
from dotenv import load_dotenv # Make sure this is imported at the top
import json # For pretty printing

# --- Load Environment Variables ---
load_dotenv()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
if GEMINI_API_KEY:
    print(f"Successfully loaded Gemini API Key starting with: {GEMINI_API_KEY[:5]}...")
else:
    print("!!! Gemini API Key NOT found after os.getenv() !!!")
    exit()

# --- Gemini LLM Configuration ---
try:
    genai.configure(api_key=GEMINI_API_KEY)
    # Using the latest recommended fast model
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    print("Successfully configured Gemini model.")
except Exception as e:
    print(f"Error configuring Gemini: {e}")
    exit()
# --- End Gemini Configuration ---


# --- Function to Parse LLM Response ---
# (Keep your existing parse_song_attributes function as it was)
def parse_song_attributes(text):
    attributes = {
        'bpm': 'N/A',
        'key': 'N/A',
        'frequency_emphasis': 'N/A',
        'rhythm_feel': 'N/A',
        'main_instruments': 'N/A',
        'llm_raw_response': text # Store raw response for debugging
    }
    if not text or "Error:" in text: # Also check if the text contains our error message
        attributes['llm_raw_response'] = text if text else "No response received."
        return attributes

    # Use re.search with flags for case-insensitivity (re.IGNORECASE) and multiline (re.MULTILINE)
    # Use non-greedy matching (.*?) to capture text until the next label or end of line

    bpm_match = re.search(r"bpm:?\s*(.*?)(?:\n|$)", text, re.IGNORECASE | re.MULTILINE)
    if bpm_match: attributes['bpm'] = bpm_match.group(1).strip()

    key_match = re.search(r"key:?\s*(.*?)(?:\n|$)", text, re.IGNORECASE | re.MULTILINE)
    if key_match: attributes['key'] = key_match.group(1).strip()

    freq_match = re.search(r"frequency.*?range.*?emphasis:?\s*(.*?)(?:\n|$)", text, re.IGNORECASE | re.MULTILINE)
    if freq_match: attributes['frequency_emphasis'] = freq_match.group(1).strip()

    rhythm_match = re.search(r"rhythm.*?feel.*?:?\s*(.*?)(?:\n|$)", text, re.IGNORECASE | re.MULTILINE)
    if rhythm_match: attributes['rhythm_feel'] = rhythm_match.group(1).strip()

    instruments_match = re.search(r"main instruments.*?:?\s*(.*?)(?:\n|$)", text, re.IGNORECASE | re.MULTILINE)
    if instruments_match: attributes['main_instruments'] = instruments_match.group(1).strip()

    # Simple check if the LLM indicated it doesn't know
    # Make sure this doesn't overwrite valid data if some fields were found
    if re.search(r"don't have information|cannot provide specific|unable to find details", text, re.IGNORECASE):
        print("  LLM indicated lack of specific information for some attributes.")
        # Only mark as Unknown if they weren't already parsed successfully
        attributes['bpm'] = attributes['bpm'] if attributes['bpm'] not in ['N/A', '', None] else "Unknown"
        attributes['key'] = attributes['key'] if attributes['key'] not in ['N/A', '', None] else "Unknown"
        attributes['frequency_emphasis'] = attributes['frequency_emphasis'] if attributes['frequency_emphasis'] not in ['N/A', '', None] else "Unknown"
        attributes['rhythm_feel'] = attributes['rhythm_feel'] if attributes['rhythm_feel'] not in ['N/A', '', None] else "Unknown"
        attributes['main_instruments'] = attributes['main_instruments'] if attributes['main_instruments'] not in ['N/A', '', None] else "Unknown"

    return attributes
# --- End Parsing Function ---

# --- Assume playlist_songs is loaded from previous steps ---
# Example placeholder:
# playlist_songs = [
#    {'name': 'Song 1', 'artist': 'Artist 1', 'album': 'Album 1'},
#    {'name': 'Song 2', 'artist': 'Artist 2', 'album': 'Album 2'},
# ]
# Make sure this variable exists and contains your Spotify data!
if 'playlist_songs' not in locals():
     print("Error: 'playlist_songs' variable not found. Make sure Spotify data is loaded.")
     exit()

# --- Process Songs and Query LLM ---
song_attributes_list = []
print(f"\n--- Querying Gemini for attributes of {len(playlist_songs)} songs ---")

for index, song in enumerate(playlist_songs):
    print(f"Processing song {index + 1}/{len(playlist_songs)}: {song['artist']} - {song['name']}")

    # Construct the prompt (same as before)
    prompt = f"""
Analyze the musical attributes of the song "{song['name']}" by "{song['artist']}" from the album "{song['album']}".

Provide the following information if available in your knowledge base. Please format the answer clearly with labels for each attribute:
1. Estimated BPM (Beats Per Minute): [Provide value or range]
2. Key (e.g., C Major, A Minor): [Provide key]
3. Typical Frequency Range Emphasis (e.g., Bass-heavy, Mid-range focused, Bright highs, Balanced): [Describe emphasis]
4. Primary Rhythmic Feel/Time Signature (e.g., 4/4 Straight Rock, 12/8 Shuffle, 3/4 Waltz, Funky Syncopation): [Describe rhythm]
5. Main Instruments Used: [List instruments]
"""

    # --- Start: Modified Retry Loop ---
    llm_response_text = None
    attempts = 0
    max_attempts = 4 # Increased max attempts slightly
    retry_statuses = {400, 429, 500, 503} # Status codes worth retrying

    while attempts < max_attempts:
        try:
            # Send prompt to Gemini
            response = model.generate_content(prompt)
            llm_response_text = response.text
            print(f"  Received response from LLM.")
            # Check for safety blocks or empty responses right away
            if not llm_response_text:
                 # Check response.prompt_feedback for safety reasons
                 if hasattr(response, 'prompt_feedback') and response.prompt_feedback.block_reason:
                     print(f"  Response blocked due to safety settings: {response.prompt_feedback.block_reason}")
                     llm_response_text = f"Error: Response blocked by safety filter ({response.prompt_feedback.block_reason})"
                 else:
                     print("  Received empty response from LLM.")
                     llm_response_text = "Error: Received empty response"
                 # No point retrying if blocked or empty, break the inner loop
                 break
            # Check if the response indicates lack of info explicitly (might avoid unnecessary parsing)
            # This check is now also in parse_song_attributes, but can be useful here too
            # if re.search(r"don't have information|cannot provide specific|unable to find details", llm_response_text, re.IGNORECASE):
            #    print("  LLM response indicates lack of specific information (pre-parsing).")

            break # Success, exit retry loop

        except Exception as e:
            attempts += 1
            error_handled_for_retry = False
            error_status_code = None

            # Try to get status code specifically from google.api_core.exceptions
            if hasattr(e, 'status_code'): # Specific check for google api errors
                 error_status_code = e.status_code
            elif hasattr(e, 'code') and callable(e.code): # Check for gRPC errors
                 try:
                      error_status_code = e.code().value[0] # gRPC status codes are often tuples
                 except:
                      pass # Ignore if we can't get gRPC code

            # --- Retry Logic ---
            if error_status_code and error_status_code in retry_statuses:
                 print(f"  API Error (Status {error_status_code}, Attempt {attempts}/{max_attempts}): {e}. Retrying...")
                 error_handled_for_retry = True
                 time.sleep(5 * attempts) # Exponential backoff

            # Check specifically for the API Key error text in the exception string
            elif "API Key not found" in str(e) or "API_KEY_INVALID" in str(e):
                 print(f"  API Key Error suspected (Attempt {attempts}/{max_attempts}): {e}. Retrying...")
                 error_handled_for_retry = True
                 time.sleep(5 * attempts) # Exponential backoff

            else:
                 # Generic or non-retryable error
                 print(f"  Non-retryable Error querying Gemini (Attempt {attempts}/{max_attempts}): {e}")
                 # Log this error differently if needed

            # --- Handling after loop attempt ---
            if attempts >= max_attempts:
                print(f"  >>>>> FAILED PROCESSING (Max Retries): {song['artist']} - {song['name']} <<<<<")
                llm_response_text = f"Error: Failed after {max_attempts} attempts. Last error: {e}"
                break # Exit retry loop after max attempts

            elif not error_handled_for_retry:
                # If it was a generic error we didn't specifically handle for retry
                print(f"  >>>>> FAILED PROCESSING (Unexpected Error): {song['artist']} - {song['name']} <<<<<")
                llm_response_text = f"Error: Unexpected error during query: {e}"
                break # Exit retry loop

    # --- End: Modified Retry Loop ---

    # Parse the response (even if it's an error message, the parser handles it)
    attributes = parse_song_attributes(llm_response_text)

    # Combine original song info with new attributes
    song_data = song.copy() # Start with original name, artist, album
    song_data.update(attributes) # Add the parsed attributes

    song_attributes_list.append(song_data)

    # --- IMPORTANT: Rate Limiting ---
    # Pause between requests to avoid hitting API limits
    print("  Pausing before next request...")
    time.sleep(10) # Increased sleep time

print("\n--- Finished fetching attributes ---")

# --- Display Results ---
if song_attributes_list:
    print("\n--- Song Attributes Collected ---")
    # Pretty print the list of dictionaries
    print(json.dumps(song_attributes_list, indent=2))
    print("-----------------------------")

    # --- Optional: Summarize Failures ---
    failures = [s for s in song_attributes_list if "Error:" in s.get('llm_raw_response', '')]
    if failures:
        print(f"\n--- Summary: {len(failures)} songs encountered errors ---")
        for failed_song in failures:
             print(f"  - {failed_song['artist']} - {failed_song['name']}: {failed_song['llm_raw_response'][:100]}...") # Print start of error
        print("----------------------------------------")
    else:
        print("\n--- Summary: All songs processed successfully (though some attributes might be unknown). ---")

else:
    print("No attribute data was collected.")

Successfully loaded Gemini API Key starting with: AIzaS...
Successfully configured Gemini model.

--- Querying Gemini for attributes of 37 songs ---
Processing song 1/37: Queen - We Will Rock You - Remastered 2011
  API Key Error suspected (Attempt 1/4): 400 API Key not found. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API Key not found. Please pass a valid API key."
]. Retrying...
  API Key Error suspected (Attempt 2/4): 400 API Key not found. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API Key not found. Please pass a valid API key."
]. Retrying...
  API Key Error suspected (Attempt 3/4): 400 API Key not found. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key:

KeyboardInterrupt: 